Vou fazer esssa bosta aqui

Quero ver se não funciona!

In [11]:
import pandas as pd
from pathlib import Path

## Classe Vazoes

Yanase

In [1]:
# -*- coding: utf-8 -*-
"""Arquivo vazoes.txt."""
from pathlib import Path
from typing import Optional, Union

import pandas as pd
import numpy as np


def ler_arquivo_vazoes_txt(arquivo: Union[str, Path]) -> pd.DataFrame:
    """
    Leitura de um arquivo txt de vazões como dataframe.

    O dataframe é semelhante ao formato original de linhas e colunas.
    Como índice ficam o posto e ano, e como colunas os meses.

    Parameters
    ----------
    posto : int
        Número do posto com o qual foi feita a correlação.

    Returns
    -------
    DataFrame
        Dataframe com os dados lidos.

    """
    # Leitura do arquivo como dataframe, usando espaços como separador.
    df_arq = pd.read_csv(arquivo, header=None, delim_whitespace=True)
    # Nomeação das colunas e índices
    df_arq.columns = ['posto', 'ano', *range(1, 13)]
    df_arq.set_index(['posto', 'ano'], inplace=True)
    df_arq.columns.name = 'mes'

    df_arq = df_arq.astype(int)

    return df_arq


def _periodizar_df_arq(df_arq: pd.DataFrame) -> pd.DataFrame:
    """
    Cria um dataframe 'periodizado' (ano/mes x postos).

    Com o tempo como linhas e os postos como colunas, tem-se um dataframe
    mais simples com apenas duas dimensões, facilitando sua manipulação.

    Parameters
    ----------
    df_arq : DataFrame
        Dataframe com os dados lidos, oriundo de ler_arquivo_vazoes_txt.

    Returns
    -------
    DataFrame
        Dataframe com os dados 'periodizados'.

    """
    # Empilha os meses e desempilha o posto
    df_period = df_arq.stack().unstack('posto')

    # Transforma os indexes ano e mes em um index só que representa ambas
    # as informações de uma melhor forma. O nome do index continua sendo 'mes'
    df_period.index = pd.PeriodIndex(df_period.index.get_level_values('ano').astype(str)
                                   + '/'
                                   + df_period.index.get_level_values('mes').astype(str),
                                   freq='M')
    df_period.index.name = 'mes'

    # Se todas as colunas são 0, muda para NaN
    # Presume-se aqui que isso só acontece nos períodos finais, caso
    # contrário melhorar este trecho.
    df_period.loc[(df_period == 0).all(axis='columns')] = np.NaN
    # exclui as colunas com NaN
    df_period.dropna(inplace=True)
    # Sem NaN, é possível deixar as colunas com o tipo inteiro
    df_period = df_period.astype(int)

    return df_period


def _desperiodizar_df_arq(df_period: pd.DataFrame) -> pd.DataFrame:
    """
    Cria um dataframe 'desperiodizado', voltando ao formato tradicional.

    Parameters
    ----------
    df_period : DataFrame
        Dataframe com os dados 'periodizados'.

    Returns
    -------
    DataFrame
        Dataframe com os dados 'desperiodizados'.

    """
    df_arq = df_period.set_index([df_period.index.year,
                                df_period.index.month])
    df_arq.index.names = ['ano', 'mes']
    df_arq = df_arq.stack().unstack('mes').swaplevel()

    # Substitui os NaN por 0 e deixa como inteiro
    df_arq.fillna(0, inplace=True)
    df_arq = df_arq.astype(int)

    df_arq.sort_index(inplace=True)

    return df_arq


class VazoesTxt:
    """
    Classe que representa o arquivo vazoes.txt.

    Attributes
    ----------
    df_arquivo : DataFrame
        Representação mais fiel ao estilo do arquivo.
    df_periodizado : DataFrame
        Representação (ano/mes x posto). Simplifica cálculos e manipulações
        por ter apenas duas dimensões.

    """

    def __init__(self, arquivo: Optional[Union[str, Path]] = None):
        """
        Construtor do objeto.

        Parameters
        ----------
        arquivo : str or Path, optional
            Nome ou caminho do arquivo de vazões.txt utilizado.

        """
        self._filepath = Path(arquivo) if arquivo else None

        # Construção dos dataframes.
        # Poderiam ser tanto métodos como funções à parte,
        # discutir o que seria melhor
        self.df_arquivo = ler_arquivo_vazoes_txt(self._filepath) if arquivo else None
        self.df_period = _periodizar_df_arq(self.df_arquivo) if arquivo else None


    @classmethod
    def from_df_period(cls, df_period: pd.DataFrame):
        """
        Construtor a partir de um dataframe 'periodizado'.

        Parameters
        ----------
        df_period : DataFrame
            Dataframe com os dados 'periodizados' (PeriodIndex freq 'M').

        """
        self = cls()
        self.df_period = df_period
        self.df_arquivo = _desperiodizar_df_arq(self.df_period)

        return self

    def add_novo_periodo(self,
                         df_new_months: pd.DataFrame,
                         inplace: bool = False,
                         ) -> None:
        """
        Adiciona informações dos próximos meses.

        Parameters
        ----------
        df_new_months : DataFrame
            Dataframe com as informações a serem adicionadas.

        Returns
        -------
        if not inplace:
            VazoesTxt
                Um novo objeto com as informações concatenadas
        if inplace:
            None
                O atributos do objeto são modificados localmente.

        """
        # Verifica o próximo mês a ser preenchido
        next_month = self.df_period.index[-1] + pd.offsets.MonthEnd()
        # Calcula a diferença entre o próximo mês e o primeiro do período informado
        diff_months = next_month - df_new_months.index[0]
        # Ajusta o index
        df_new_months_ajust = df_new_months.copy()
        df_new_months_ajust.index = df_new_months.index + diff_months

        # Concatena com os dados originais
        df_new = pd.concat([self.df_period, df_new_months_ajust],
                            verify_integrity=True)

        # Se não muda localmente, retorna um novo objeto
        if inplace is False:
            return self.from_df_period(df_new)

        # Muda o objeto no local
        self.df_period = df_new
        # Desnormaliza
        self.df_arquivo = _desperiodizar_df_arq(self.df_period)

        return None


    def salvar_txt(self,
                   arquivo_destino: Union[str, Path],
                   ) -> None:
        """
        Salva o dataframe modificado em um novo arquivo de vazões.txt.

        Parameters
        ----------
        arquivo_destino : str or Path
            Nome ou caminho do arquivo de destino.

        """
        lista_linhas = list()
        for idx, row in self.df_arquivo.iterrows():
            lista_linhas.append(f"{idx[0]:3} {idx[1]:4}"
                                f"{row[1]:6}{row[2]:6}{row[3]:6}"
                                f"{row[4]:6}{row[5]:6}{row[6]:6}"
                                f"{row[7]:6}{row[8]:6}{row[9]:6}"
                                f"{row[10]:6}{row[11]:6}{row[12]:6}")

        conteudo_arquivo = '\n'.join(lista_linhas)

        with open(arquivo_destino, 'w') as file:
            file.write(conteudo_arquivo)
            file.write('\n')

## Cálculo da Correlação

def calc_corr_last12(df_hist: pd.DataFrame) -> pd.DataFrame:
    """
    Cálculo da correlação dos últimos 12 meses informados com anos anteriores.

    Parameters
    ----------
    df_hist : DataFrame
        Dataframe do histórico a ser analisado.
        As 12 últimas linhas devem ser os 12 meses com os quais será calculada
        a correlação com dados de anos anteriores.

    Returns
    -------
    DataFrame
        Dataframe com as correlações calculadas.
        Como index é informado o mês final do período de 12 meses com o qual
        foi calculada a correlação.

    """
    # pylint: disable=protected-access
    # Conferindo se o index está no formato desejado (pd.PeriodIndex freq=M)
    if not isinstance(df_hist.index.freq, pd._libs.tslibs.offsets.MonthEnd):
        raise Exception("Favor informar dataframe periodizado mensalmente.")

    df_ref = df_hist.copy()
    # Trecho de 12 meses com o qual será calculada a correlação
    df_trecho = df_ref.iloc[-12:]
    # Espalha o trecho por uma cópia do dataframe respeitando os meses
    df_other = df_ref.copy()
    for month in range(1, 13):
        df_other[df_other.index.month == month] = df_trecho[df_trecho.index.month == month].iloc[0]

    # Calcula a correlação
    df_corr = df_ref.rolling(12).corr(df_other)
    # Filtra os dados da correlação com o mês final
    df_corr = df_corr[df_corr.index.month == df_trecho.index.month[-1]]
    df_corr.index.name = 'mes_final'

    return df_corr

## Modelo

In [8]:
import pandas as pd
from typing import Optional

class ModeloCamila:
    """
    Classe que representa o um modelo de correlações simples e Teste da amplitude.
    """

    def __init__(self,
                 coluna: int,
                 posicao: int = 1,
                 ):
        """
        Criação do modelo.

        """
        self.coluna = coluna
        self.posicao = posicao

        # Atributos que serão definidos apenas ao realizar o fit
        self.df_base = None
        self.df_correlacao = None
        self._period_ = None
        self.correlacao_ = None
        self.correlacao_outros_ = None
        self.mes_final_periodo_ = None


    def fit(self, df_base: pd.DataFrame) -> 'ModeloCamila':
        """
        Cálculo das correlações.
        """
        self.df_base = df_base.copy()
        self.df_correlacao = calc_corr_last12(df_base)

        top_corr = self.df_correlacao[self.coluna].sort_values(ascending=False)

        # Dados que serão usados para previsão
        self._period_ = top_corr.index[self.posicao]
        #======================================================================
        mes_escolhido_ini = self._period_ + 1        
        
        #série com a primeiro mês da previsão extendida
        series_proximo_mes = self.df_base.loc[mes_escolhido_ini]
        #Renomeando index
        series_proximo_mes.name = self.df_base.index[-1] + 1
        #print(series_proximo_mes.to_frame().T)
        
        #Acrescentando o primeiro mês da previsão extendida nos dados originais
        df_base_modif = pd.concat([self.df_base, series_proximo_mes.to_frame().T])
        #print(df_base_modif)
        print(self._period_)
        print(mes_escolhido_ini)
        
        #Cálculo da razão
        previsao_refinitiv = self.df_base.iloc[-1]
        previsao_extendida = df_base_modif.iloc[-1]
        df_razao = previsao_refinitiv / previsao_extendida
        
        #Usando a função shift
        df_teste = df_base_modif.shift(periods=0, freq='1M')
        
        print(df_teste)
        
        #======================================================================
        # Outras informações sobre o modelo 'treinado'
        # Usando sufixo _ semelhante ao scikit-learn
        self.correlacao_ = top_corr.iloc[self.posicao]
        self.correlacao_outros_ = self.df_correlacao.loc[top_corr.index[self.posicao]]
        self.mes_final_periodo_ = self._period_.strftime('%Y-%m')

        # Retorna o próprio objeto para o caso de ser encadeado com .predict()
        return self


    def predict(self,
                num_meses: Optional[int] = None,
                ) -> pd.DataFrame:
        """
        Previsão para os próximos 'num_meses'.

        """
        # Mês seguinte ao período selecionado pelo modelo
        try:
            # Não havendo erro supomos que o fit foi realizado
            mes_escolhido_ini = self._period_ + 1
        except TypeError:
            # Podemos futuramente usar o erro sklearn.exceptions.NotFittedError,
            # mas para simplificar:
            raise Exception("Realizar o fit do modelo antes") from None

        # Primeiro mês a ser previsto
        next_month = self.df_base.index[-1] + 1

        # Se num_meses não for informado vai o final do ano
        num_meses = num_meses if num_meses else (13 - mes_escolhido_ini.month)

        # Dados do período escolhido
        df_escolhido = self.df_base.loc[mes_escolhido_ini:].iloc[:num_meses]

        # Ajusta o index para o novo período
        df_trecho_ajust = df_escolhido.copy()
        df_trecho_ajust.index = pd.period_range(next_month,
                                                periods=len(df_trecho_ajust),
                                                freq='M')

        return df_trecho_ajust


    def teste_amplitude(self):
        """Verifica se a razão entre os meses de previsão
        estão em um intervalo entre o razão mínina e a máxima do histórico.
        
        Caso não no intervalo 
        """
        #-------------
        #Teste da Amplitude
        
        #Cria uma razão com o último ano de previsão da Refinitiv pelo primeiro mês da previsão extendida
        # df_trecho = self.predict()
        
        df_trecho = self.predict()
        #Desperiodizar o index para fazer o cálculo da razão
       
        #dict_amplitude = {}
        df_original = pd.DataFrame(self.df_base)
        df_razao = df_trecho.describe()
        #df_razao = df_original.index[-1].values
        
        # #Resultado da razão em todos os postos, matriz de anos por posto
        # self.df_amplitude = df_razao
        # self.df_amplitude = df_razao.unstack('posto')
        
        # #Seleciona a amplitude do último ano
        # self.df_amplitude_previsao = self.df_amplitude.iloc[-1:].squeeze()
        # self.df_amplitude_previsao = self.df_amplitude_previsao.T        
        
        # #Drop na linha com as previsões para comparar com as demais
        # self.df_amplitude = self.df_amplitude.drop(self.df_amplitude.index[-1])
        
        # #Seleciona a maior e a menor razão do histórico
        # self.df_amplitude_maximo = self.df_amplitude.max(skipna=False)
        # self.df_amplitude_minima = self.df_amplitude.min(skipna=False)
        
        # #Verifica se a razão das previsões está no intervalo entre a máxima e a mínima do histórico
        # self.verifica_max = self.df_amplitude_previsao >= self.df_amplitude_maximo 
        # self.verifica_min = self.df_amplitude_previsao <= self.df_amplitude_minima
        
        
        # if verifica_max.iloc[self.coluna] is True:
        #     self.posicao += 1
        # if verifica_min.iloc[self.coluna] is True:
        #     self.posicao += 1
        return df_original


## Teste

In [9]:

arquivo = Path('C:/Workspace/cenarios_por_correlacao_de_vazoes/teste_arquivos_entrada/VAZOES-P50.txt')

# Criação do objeto de vazões
vazoes = VazoesTxt(arquivo)

model = ModeloCamila(coluna=6, posicao=1)
# Parametrização do modelo
model.fit(vazoes.df_period)
# Predição para um novo período
novo_trecho = model.predict()

# Junção do período do arquivo com o período previsto
vazoes_new = vazoes.add_novo_periodo(novo_trecho)
# Salva novo arquivo de vazões no formato txt

# Apenas para visualização das correlações do principais postos
#correlacoes_principais = model.df_correlacao[POSTOS_PRINCIPAIS.keys()]
#correlacoes_principais.columns = POSTOS_PRINCIPAIS.values()

#----------------------------------------------------------

#a1 = a1.loc[0]


1967-04
1967-05
posto    1    2     6     7     8     9     10    11    12   14   ...  304  \
1931-01  178  178  1476  1690  1737  1759  1819  1939  2210  152  ...    0   
1931-02  371  371  2964  3318  3385  3408  3483  3661  4115  220  ...    5   
1931-03  326  326  2167  2471  2532  2558  2632  2788  3158  143  ...    0   
1931-04  479  479  1585  1827  1879  1903  1969  2100  2393  107  ...    0   
1931-05  332  332  1254  1428  1463  1479  1522  1614  1828   60  ...    0   
...      ...  ...   ...   ...   ...   ...   ...   ...   ...  ...  ...  ...   
2023-01  253  253  1611  1797  1829  1846  1899  2011  2241   76  ...    0   
2023-02  236  236  1541  1727  1776  1791  1850  1965  2226   75  ...    0   
2023-03  197  197  1304  1482  1527  1546  1598  1704  1948   65  ...    0   
2023-04   93   93   574   661   684   694   722   775   902   34  ...    0   
2023-05  108  108   880  1003  1028  1039  1071  1136  1288   45  ...    0   

posto    306  313   314  315  316  317  318  31

In [4]:
a1 = model.predict()

In [5]:
a2 = model.teste_amplitude()